In [1]:
import requests
import numpy as np
np.set_printoptions(precision=2)
import pandas as pd
import math

import json
import time
from datetime import datetime, timedelta
import sgp4
from sgp4.api import Satrec, WGS72
from sgp4.conveniences import jday
from sgp4.ext import invjday

from sgp4.api import accelerated

import matplotlib.pyplot as plt

from astropy.time import Time

from astropy.coordinates import TEME, CartesianDifferential, CartesianRepresentation
from astropy import units as u
from astropy.coordinates import ITRS

from astropy.coordinates import EarthLocation, AltAz

from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
def get_tle(sat_num):
    # Space-Track API의 URL
    url = "https://www.space-track.org/ajaxauth/login"

    # 요청에 필요한 인증 정보
    payload = {"identity": 'jybyun@hanyang.ac.kr', "password": ''}

    # Session 객체 생성
    session = requests.Session()

    # 로그인 요청
    response = session.post(url, data=payload)

    if response.status_code != 200:
        print("로그인에 실패했습니다.")
        return None

    # TLE 요청
    response = session.get(
        f"https://www.space-track.org/basicspacedata/query/class/tle/NORAD_CAT_ID/{sat_num}/EPOCH/2022-12-31--2024-08-01/orderby/EPOCH%20asc/format/3le/emptyresult/show"
    )#

    return response

In [3]:
col_name = ['time',
            'x','y','z',
            'vx', 'vy', 'vz',
            'altitude', 'velocity', 
            'x_earth','y_earth','z_earth',
            'vx_earth','vy_earth','vz_earth',
            'apogee', 'perigee', 
            'inclination', 'eccentricity', 'raan',
            'lon','lat','height']

In [ ]:
no = 55841

In [ ]:
response = get_tle(no)
tle_datas = response.text.split('\r\n')

tle_first_lst = []
tle_second_lst = []

for i in range(0, len(tle_datas)-2, 3):
    tle_first_lst.append(tle_datas[i+1])
    tle_second_lst.append(tle_datas[i+2])

tle_df = pd.DataFrame({'first_line':tle_first_lst, 'second_line':tle_second_lst})
tle_df.drop_duplicates(keep='first', inplace=True)
tle_df.reset_index(inplace=True, drop=True)

In [ ]:
change_times = []
for i in range(len(tle_df)):
    tle_1, tle_2 = tle_df['first_line'].iloc[i], tle_df['first_line'].iloc[i]
    satellite = Satrec.twoline2rv(tle_1,tle_2)
    jdsatepoch = satellite.jdsatepoch
    jdsatepochfrac = satellite.jdsatepochF
    epochdatetime = invjday(jdsatepoch + jdsatepochfrac)
    change_times.append(epochdatetime)
    
change_df = pd.DataFrame(change_times)
change_df.columns = ['year', 'month', 'day', 'hour', 'minute', 'second']
change_df['first_line']=tle_df['first_line']
change_df['second_line']=tle_df['second_line']
change_df['time'] = change_df['year'].astype('str') + '-' + change_df['month'].astype('str') + '-' + change_df['day'].astype('str') + ' ' + change_df['hour'].astype('str') + ':' + change_df['minute'].astype('str') + ':' + change_df['second'].astype('int').astype('str')
change_df['time'] = pd.to_datetime(change_df['time'])

change_df = change_df.loc[change_df.sort_values(by=['year','month','day','hour','minute','second'])[['year','month','day','hour','minute']].drop_duplicates(keep='last').index].copy()
change_df.reset_index(inplace=True, drop=True)

end_point = change_df.iloc[-1:].copy()

end_point['month'] = 8
end_point['day'] = 1
end_point['hour'] = 0
end_point['minute'] = 0
end_point['second'] = 0

end_point['time'] = pd.to_datetime(end_point['year'].astype('str') + '-' + end_point['month'].astype('str') + '-' + end_point['day'].astype('str') + ' ' + end_point['hour'].astype('str') + ':' + end_point['minute'].astype('str') + ':' + end_point['second'].astype('int').astype('str'))

change_df = pd.concat([change_df,end_point], axis=0).copy()
change_df.reset_index(inplace=True, drop=True)

day_change_idx = list(change_df[['year','month','day']].drop_duplicates().index)

In [ ]:

all_info = []

for i in tqdm(range(len(change_df)-1)):

    sat_info = change_df.iloc[i]
    next_sat_info = change_df.iloc[i+1]

    tle_1, tle_2 = sat_info['first_line'], sat_info['second_line']

    satellite = Satrec.twoline2rv(tle_1,tle_2)
    jdsatepoch = satellite.jdsatepoch
    jdsatepochfrac = satellite.jdsatepochF
    epochdatetime = list(invjday(jdsatepoch + jdsatepochfrac))
    epochdatetime[-1] = int(epochdatetime[-1])

    next_tle_1, next_tle_2 = next_sat_info['first_line'], next_sat_info['second_line']

    next_satellite = Satrec.twoline2rv(next_tle_1,next_tle_2)
    next_jdsatepoch = next_satellite.jdsatepoch
    next_jdsatepochfrac = next_satellite.jdsatepochF
    next_epochdatetime = list(invjday(next_jdsatepoch + next_jdsatepochfrac))
    next_epochdatetime[-1] = int(next_epochdatetime[-1])


    earth_radius = 6378.137
    rad2deg = 180.0 / 3.141592653589793

    start = datetime(*epochdatetime)

    if i == len(change_df)-2:
        next_epochdatetime = [2024, 8, 1, 0, 0, 0]
        end = datetime(*next_epochdatetime)
    else:
        end = datetime(*next_epochdatetime)
        end += timedelta(seconds=1)

    start += timedelta(minutes=1)
    start -= timedelta(seconds=start.second)

    epoch = start

    jd_lst = []
    fr_lst = []

    time_lst = []

    while( epoch < end ):

        year = epoch.year
        month = epoch.month
        date = epoch.day
        hour = epoch.hour
        minute = epoch.minute
        second = epoch.second

        time_lst.append(epoch)

        jd, fr = jday(year, month, date, hour, minute, second)

        jd_lst.append(jd)
        fr_lst.append(fr)
        
        epoch += timedelta(minutes=1)

    e, r, v = satellite.sgp4_array(np.array(jd_lst), np.array(fr_lst))

    t_lst = Time(list(np.array(jd_lst) + np.array(fr_lst)), format='jd')

    teme_p = CartesianRepresentation(r[:,0]*u.km, r[:,1]*u.km, r[:,2]*u.km)
    teme_v = CartesianDifferential(v[:,0]*u.km/u.s,v[:,1]*u.km/u.s,v[:,2]*u.km/u.s)
    teme = TEME(teme_p.with_differentials(teme_v), obstime=t_lst)
    itrs_geo = teme.transform_to(ITRS(obstime=t_lst))
    locations = itrs_geo.earth_location
    geodetic_coords = locations.geodetic

    all_info.append(pd.DataFrame([time_lst,
              r[:,0],r[:,1],r[:,2],
              v[:,0],v[:,1],v[:,2],
              (r[:,0]**2 + r[:,1]**2 + r[:,2]**2)**0.5,(v[:,0]**2 + v[:,1]**2 + v[:,2]**2)**0.5,
              teme_p.x.value, teme_p.y.value, teme_p.z.value,
              teme_v.d_x.value,teme_v.d_y.value,teme_v.d_z.value,
              [satellite.alta * earth_radius] * len(time_lst), [satellite.altp * earth_radius]* len(time_lst),
              [satellite.inclo*rad2deg] * len(time_lst), [satellite.ecco]* len(time_lst),[satellite.nodeo]* len(time_lst),
              geodetic_coords.lon.value, geodetic_coords.lat.value,geodetic_coords.height.value,
                ]))

result = pd.concat(all_info,axis=1).T

result.columns = col_name
result.time = pd.to_datetime(result.time)
result = result.set_index(['time'])

In [ ]:
no_lst = [17122, 29043, 29505, 32791, 36596, 
          37337, 38345, 39227, 41850, 42841, 
          43186, 43807, 44343, 44349, 44350,
          44351, 44353, 44358, 46267, 49962, 
          55909, 58018, 58019, 58023, 58024, 
          58464]

In [ ]:
no_lst = [43823,45246,29349,37265,42691,42984]

In [ ]:
no_lst.sort()
no_lst

In [4]:
no_lst = [43823, 45246, 29349, 37265, 42691, 42984, 55841, 39227, 44343, 44349, 44350, 44351, 44353, 44358, 58464, 46267, 53611, 53019, 51961, 47775, 51969, 58722, 56361, 56289, 56783, 48018]


In [5]:
for no in no_lst:
    print(no)
    
    response = get_tle(no)
    tle_datas = response.text.split('\r\n')

    tle_first_lst = []
    tle_second_lst = []

    for i in range(0, len(tle_datas)-2, 3):
        tle_first_lst.append(tle_datas[i+1])
        tle_second_lst.append(tle_datas[i+2])

    tle_df = pd.DataFrame({'first_line':tle_first_lst, 'second_line':tle_second_lst})
    tle_df.drop_duplicates(keep='first', inplace=True)
    tle_df.reset_index(inplace=True, drop=True)

    change_times = []
    for i in range(len(tle_df)):
        tle_1, tle_2 = tle_df['first_line'].iloc[i], tle_df['first_line'].iloc[i]
        satellite = Satrec.twoline2rv(tle_1,tle_2)
        jdsatepoch = satellite.jdsatepoch
        jdsatepochfrac = satellite.jdsatepochF
        epochdatetime = invjday(jdsatepoch + jdsatepochfrac)
        change_times.append(epochdatetime)
        
    change_df = pd.DataFrame(change_times)
    change_df.columns = ['year', 'month', 'day', 'hour', 'minute', 'second']
    change_df['first_line']=tle_df['first_line']
    change_df['second_line']=tle_df['second_line']
    change_df['time'] = change_df['year'].astype('str') + '-' + change_df['month'].astype('str') + '-' + change_df['day'].astype('str') + ' ' + change_df['hour'].astype('str') + ':' + change_df['minute'].astype('str') + ':' + change_df['second'].astype('int').astype('str')
    change_df['time'] = pd.to_datetime(change_df['time'])

    change_df = change_df.loc[change_df.sort_values(by=['year','month','day','hour','minute','second'])[['year','month','day','hour','minute']].drop_duplicates(keep='last').index].copy()
    change_df.reset_index(inplace=True, drop=True)

    end_point = change_df.iloc[-1:].copy()

    end_point['month'] = 8
    end_point['day'] = 1
    end_point['hour'] = 0
    end_point['minute'] = 0
    end_point['second'] = 0

    end_point['time'] = pd.to_datetime(end_point['year'].astype('str') + '-' + end_point['month'].astype('str') + '-' + end_point['day'].astype('str') + ' ' + end_point['hour'].astype('str') + ':' + end_point['minute'].astype('str') + ':' + end_point['second'].astype('int').astype('str'))

    change_df = pd.concat([change_df,end_point], axis=0).copy()
    change_df.reset_index(inplace=True, drop=True)


    all_info = []

    for i in tqdm(range(len(change_df)-1)):

        sat_info = change_df.iloc[i]
        next_sat_info = change_df.iloc[i+1]

        tle_1, tle_2 = sat_info['first_line'], sat_info['second_line']

        satellite = Satrec.twoline2rv(tle_1,tle_2)
        jdsatepoch = satellite.jdsatepoch
        jdsatepochfrac = satellite.jdsatepochF
        epochdatetime = list(invjday(jdsatepoch + jdsatepochfrac))
        epochdatetime[-1] = int(epochdatetime[-1])

        next_tle_1, next_tle_2 = next_sat_info['first_line'], next_sat_info['second_line']

        next_satellite = Satrec.twoline2rv(next_tle_1,next_tle_2)
        next_jdsatepoch = next_satellite.jdsatepoch
        next_jdsatepochfrac = next_satellite.jdsatepochF
        next_epochdatetime = list(invjday(next_jdsatepoch + next_jdsatepochfrac))
        next_epochdatetime[-1] = int(next_epochdatetime[-1])


        earth_radius = 6378.137
        rad2deg = 180.0 / 3.141592653589793

        start = datetime(*epochdatetime)

        if i == len(change_df)-2:
            next_epochdatetime = [2024, 8, 1, 0, 0, 0]
            end = datetime(*next_epochdatetime)
        else:
            end = datetime(*next_epochdatetime)
            end += timedelta(seconds=1)

        start += timedelta(minutes=1)
        start -= timedelta(seconds=start.second)

        epoch = start

        jd_lst = []
        fr_lst = []

        time_lst = []

        while( epoch < end ):

            year = epoch.year
            month = epoch.month
            date = epoch.day
            hour = epoch.hour
            minute = epoch.minute
            second = epoch.second

            time_lst.append(epoch)

            jd, fr = jday(year, month, date, hour, minute, second)

            jd_lst.append(jd)
            fr_lst.append(fr)
            
            epoch += timedelta(minutes=1)

        e, r, v = satellite.sgp4_array(np.array(jd_lst), np.array(fr_lst))

        t_lst = Time(list(np.array(jd_lst) + np.array(fr_lst)), format='jd')

        teme_p = CartesianRepresentation(r[:,0]*u.km, r[:,1]*u.km, r[:,2]*u.km)
        teme_v = CartesianDifferential(v[:,0]*u.km/u.s,v[:,1]*u.km/u.s,v[:,2]*u.km/u.s)
        teme = TEME(teme_p.with_differentials(teme_v), obstime=t_lst)
        itrs_geo = teme.transform_to(ITRS(obstime=t_lst))
        locations = itrs_geo.earth_location
        geodetic_coords = locations.geodetic

        all_info.append(pd.DataFrame([time_lst,
                r[:,0],r[:,1],r[:,2],
                v[:,0],v[:,1],v[:,2],
                (r[:,0]**2 + r[:,1]**2 + r[:,2]**2)**0.5,(v[:,0]**2 + v[:,1]**2 + v[:,2]**2)**0.5,
                teme_p.x.value, teme_p.y.value, teme_p.z.value,
                teme_v.d_x.value,teme_v.d_y.value,teme_v.d_z.value,
                [satellite.alta * earth_radius] * len(time_lst), [satellite.altp * earth_radius]* len(time_lst),
                [satellite.inclo*rad2deg] * len(time_lst), [satellite.ecco]* len(time_lst),[satellite.nodeo]* len(time_lst),
                geodetic_coords.lon.value, geodetic_coords.lat.value,geodetic_coords.height.value,
                    ]))

    result = pd.concat(all_info,axis=1).T

    result.columns = col_name
    result.time = pd.to_datetime(result.time)
    result = result.set_index(['time'])

    result.sort_values(by=['year', 'month', 'day', 'hour', 'minute', 'second']).reset_index(drop=True).to_csv('data/'+ str(no) +'.csv')

43823


 97%|█████████▋| 1519/1560 [00:24<00:00, 62.76it/s]